<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Week-2-Homework" data-toc-modified-id="Week-2-Homework-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Week 2 Homework</a></span></li></ul></div>

In [ ]:
%%bash

ls

In [ ]:
%%bash

grep -o "aaaaaa" test1.fasta| wc -l

In [ ]:
%%bash

LanguageArray=("PHP"  "Java"  "C#"  "C++"  "VB.Net"  "Python"  "Perl")
echo ${LanguageArray}{$LanguageArray}

## Week 2 Homework

Assignment 1, due Tuesday Week 2
Beginning Bash Scripting
Key concepts covered: Recognizing approaches to solving problems using linux commands. Become familiar with GitHub practices

Goals
Preparing and downloading files. Creating a bash script that tests knowledge of linux commands. Running the script and submitting it. 

Assignment
Story:

Jane has discovered 10 motifs that are associated with radiation resistance in her colony of bacteria, R.bifella. A motif is a short, recurring sequence of DNA that is associated with gene regulation. She wants to choose the motif that occurs most frequently across the bacterial genome.   However, since these experiments are costly, she wants to pick the one that has the highest number of occurrences in the bacterial genome. Which motif should she choose?

Steps:

SSH into the AWS server and create a folder called week2. I will be grading out of this directory.
Copy the motif list and the genome of the R.bifella from  /home/rbif/week2_assignment/
Create a single bash script that does the following:
Print out the number of occurrences for each motif that is found in the bacterial genome and output to a file called motif_count.txt
Create a fasta file for each motif (so 10 in total) which contains all of the genes and their corresponding sequences that have that motif. Each file should be named after the motif (ie ATG.txt) and outputted to a new directory called motifs
*Note*  Please comment your code. This should include a description of what the goal of the script is at the beginning. And then any clarifying comments for snippets of code.
You can find an example output of the assignment in /home/rbif/week2_example_output along with how comments should look like.

This is due Tuesday, July 31 at 11:59 pm. I will mark anything modified inside your assignment directory after that time as late.

Submission status
This assignment will accept submissions from Tuesday, 14 April 2020, 7:00 AM

<strong>First a more complex python task. I want to explore a little bit here and rank all of the various string options, moving until I find a sting with only one entries</strong>

In [ ]:
# Numpy is a common library,I will use it here,
import numpy as np

###
# Until I get the motif list, I am going to cook myself something up
# I am not going to use numpy for the code below since I am going to translate
# the Python code to Bash.
###

class Main:
    def __init__(self, number, min_length, max_length):
        motifs = []
        
        for _ in range(number):
            motifs.append(self.motif_generator(min_length, max_length))
        
        # Write the motifs to a file
        
        with open('motifs.txt', 'w') as f:
            for motif in motifs:
                f.write("{}\n".format(motif))
        
    def motif_generator(self, min_length, max_length):
        
        # Set the size of the random item
        size = np.random.randint(low = min_length, high = max_length)
        
        # Populate that list
        nucleotides = "".join(np.random.choice(["a", "c", "g", "t"], size))
        return nucleotides
    
if __name__ == '__main__':
    # Not really sure how large to make each motif, I suppose it doesn't matter.
    Main = Main(10, 7, 9)

In [ ]:
%%bash

ls

In [1]:
import re
import os
import pandas
from itertools import combinations, permutations

class Main:
    def __init__(self, path, motif_path, candidate_size):
        self.chromo_dict = self.read_fasta(path)
        
        # The idea of this script is to start by creating a
        # Candidates list, and build the test set from there. 
        self.candidate_size = candidate_size
        self.motif_path = motif_path
        
        self.chromo_tests = self.chromo_creator(self.candidate_size)
        
        # Also going to return the motif list in here
        self.flat_chains, self.motifs = self.chain_maker(self.chromo_tests, self.motif_path)
        
        self.chain_count = self.nucleotide_counter(self.flat_chains, 
                                                   self.chromo_dict, 
                                                   self.chromo_tests)
        
        self.motif_df = self.report(self.chain_count, self.motifs)
  
    def read_fasta(self, file_name):
        # This is not the most elegant solution
        # I might refactor this code later.
        
        # First read out the lines
        f = open(file_name)
        lines = f.read().splitlines()
        f.close()
        
        # Make a list of chromosomes from list
        chromosomes = "".join([x for x in lines if x != ""]).split(">")
        chromosomes.remove("")
        
        # Make a dictionary of chromosomes and entries. Could have done this in one line,
        # but I think this is easier to read.
                
        chromo_dict = dict()
        for chromosome in chromosomes:
            # Split terms into int and non int values
            entry = re.split(r'(\d+)', chromosome)
            chromo_dict["".join(entry[:2])] = entry[-1]
                             
        return chromo_dict

    def chromo_creator(self, index):        
        chromo_tests = dict()
        for value in range(index):
            # We need to make sure we also grab duplicates
            nucleotides = (value + 1) * ["a", "c", "g", "t"]
            pattern = set(permutations(nucleotides, (value + 1)))
            pattern = list(set("".join(i) for i in pattern))
            pattern.sort()
            chromo_tests[value + 1] = pattern
        
        return chromo_tests
        
        
    def chain_maker(self, chromo_tests, motif_path):
        # flatten chromo_tests.values() lists
        flat_chains = [item for sublist in chromo_tests.values() for item in sublist]
        
        # read into the set
        with open("motifs.txt") as f:
            motifs = f.readlines()

        # you may also want to remove whitespace characters like `\n` at the end of each line
        motifs = [x.strip() for x in content]

        
        # Add motifs into flat_chain list 
        return flat_chains + motifs, motifs
        
        
    def nucleotide_counter(self, flat_chains, chromo_dict, chromo_tests):
        
        # First flatten list and then make a dictionary with counts  
        chromo_count = dict()
        chain_count  = dict()
        for chain in flat_chains:
            chain_counter = 0
            chromo_count = dict()
            for chromosome in chromo_dict.keys():
                chromo_count[chromosome] = chromo_dict[chromosome].count(chain)
                chain_counter += chromo_count[chromosome]
            
            chromo_count["total"] = chain_counter
            chain_count[chain] = chromo_count
        
        return chain_count
        
    def report(self, chain_count, motifs):
        
        report_dict = {k: chain_count[k] for k in motifs}
        
        # Not sure if it is OK to use Pandas here, but it feels most natural
        motif_df = pd.DataFrame.from_dict(report_dict)
         
        motif_df.to_csv("complete.txt".format(motif))
        motif_df.to_csv("motif_count.txt")
        
        # Print out the .txt for each column
        for motif in motif_df.columns:
               motif_df[[motif]].to_csv("{}.txt".format(upper(motif)))
                
        
        return motif_df
    
if __name__ == '__main__':
    Main = Main("test1.fasta", "motifs.txt", 5)

FileNotFoundError: [Errno 2] No such file or directory: 'test1.fasta'

In [ ]:
Main.motif_df[["ggaagt"]].to_c

In [ ]:
bin(6)